In [1]:
import re
import pickle
import pandas as pd
import seaborn as sns

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import matplotlib.pyplot as plt
%matplotlib inline
import time

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


#driver = webdriver.Chrome(chromedriver)
chromeOptions = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images":2}
chromeOptions.add_experimental_option("prefs",prefs)
def init_driver():
    
    return driver


In [2]:
def navigate_to_detail_page(driver, link):
    driver.get(link)
    #check for "listing is unavailable" situation and get redirect link
    try:
        redirect_link = driver.find_element_by_css_selector('.cvipTopMsg > span > a').get_attribute('href')
        print("redirected")
        driver.get(redirect_link)
    except:
        return True

In [3]:
def get_sold_price_page(driver):
    possible_selectors = ['#convbidPrice','#convbinPrice','#prcIsum','#prcIsum_bidPrice','#mm-saleDscPrc','.vi-VR-cvipPrice']
    while True:
        try:
            price_element = driver.find_element_by_css_selector(possible_selectors.pop(0))
            return float(re.search(r'\$\d+\.\d+', price_element.text).group().replace('$',''))
        except:
            if len(possible_selectors) == 0:
                return None

In [4]:
def get_sold_date_time_page(driver):
    try:
        time_elem = driver.find_element_by_id('bb_tlft')
        date_string = time_elem.text
        time_string = driver.find_element_by_class_name('timeMs').text
        return pd.to_datetime(date_string + " " + time_string)
    except:
        return None

In [5]:
def get_memory_size(driver):
    try:
        XPath = "//td[@class='attrLabels' and contains(text(),'Memory Size')]/following-sibling::td/span"
        element = driver.find_element_by_xpath(XPath)
        #element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, XPath)))
        return element.text
    except:
        try:
            return re.search('(\d+MB|\d+GB)', driver.find_element_by_id('itemTitle').text).group(0)
        except:
            return None

In [6]:
def get_brand(driver):
    try:
        XPath = "//td[@class='attrLabels' and contains(text(),'Manufacturer')]/following-sibling::td/span"
        element = driver.find_element_by_xpath(XPath)
        #element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, XPath)))
        return element.text
    except:
        return None

In [7]:
def get_condition(driver):
    try:
        XPath = "//td[@class='attrLabels' and contains(text(),'Condition')]/following-sibling::td/div"
        #element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, XPath)))
        element = driver.find_element_by_xpath(XPath)
        return element.text.split(':')[0]
    except:
        try:
            XPath = "//th[contains(text(),'Condition')]/following-sibling::td/b"
            element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, XPath)))
            return element.text
        except:
            return None

In [8]:
def get_memory_type(driver):
    try:
        XPath = "//td[contains(text(),'Memory Type')]/following-sibling::td/span"
        #element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, XPath)))
        element = driver.find_element_by_xpath(XPath)
        return element.text
    except:
        try:
            return re.search('[A-z]?DDR\d+', driver.find_element_by_id('itemTitle').text).group(0)
        except:
            return None

In [9]:
def get_ebay_data(num_data_points=float("inf")):
    driver = init_driver()
    graphics_cards = []
    try:
        search_page_url="https://www.ebay.com/sch/i.html?_nkw=graphics+card&_in_kw=1&_ex_kw=&_sacat=0&LH_Sold=1&_udlo=&_udhi=&_samilow=&_samihi=&_sadis=15&_stpos=94040&_sargn=-1%26saslc%3D1&_salic=1&_sop=12&_dmd=1&_ipg=200&LH_Complete=1&_fosrp=1}"
        headers = ['sold_date_time','sold_price','memory_size','memory_type','manufacturer','condition']
        while len(graphics_cards) < num_data_points:
            if len(graphics_cards) % 100 == 0:
                driver.close()
                driver.quit()
                driver = webdriver.Chrome(chromedriver, chrome_options=chromeOptions)            
            driver.get(search_page_url)
            page_links = [listing.get_attribute("href") for listing in driver.find_elements_by_css_selector(".sresult.lvresult > .lvtitle > a ")]
            search_page_url = driver.find_element_by_css_selector(".gspr.next").get_attribute("href")
            for link in page_links:#TODO:make this concurrent for performance increase
                page_found = navigate_to_detail_page(driver, link)
                sold_price = get_sold_price_page(driver)
                sold_date_time = get_sold_date_time_page(driver)
                memory_size = get_memory_size(driver)
                brand = get_brand(driver)
                condition = get_condition(driver)
                memory_type = get_memory_type(driver)
                graphics_card = dict(zip(headers,[sold_date_time,sold_price, memory_size, memory_type, brand, condition]))
                print(len(graphics_cards))
                print(graphics_card)
                graphics_cards.append(graphics_card)
#         ebay_data = pd.DataFrame(graphics_cards)
#         ebay_data.to_pickle('ebay_data.pkl')
    except:
        print("error!")
    finally:
        ebay_data = pd.DataFrame(graphics_cards)
        ebay_data.to_pickle('ebay_data.pkl')
        driver.close()
        return ebay_data

In [10]:
ebay_data = get_ebay_data(10000)


0
{'sold_date_time': Timestamp('2018-07-18 23:38:00'), 'sold_price': 50.0, 'memory_size': '1GB', 'memory_type': None, 'manufacturer': None, 'condition': 'Used'}
1
{'sold_date_time': Timestamp('2018-07-18 23:23:00'), 'sold_price': 44.61, 'memory_size': '2GB', 'memory_type': 'DDR5', 'manufacturer': 'NVIDIA', 'condition': 'New'}
redirected
2
{'sold_date_time': Timestamp('2018-07-18 23:17:00'), 'sold_price': 34.99, 'memory_size': '256MB', 'memory_type': None, 'manufacturer': 'ATI', 'condition': 'New other (see details)'}
redirected
3
{'sold_date_time': Timestamp('2018-07-18 22:50:00'), 'sold_price': 37.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'New'}
redirected
4
{'sold_date_time': Timestamp('2018-07-18 22:44:00'), 'sold_price': 125.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'Used'}
redirected
5
{'sold_date_time': Timestamp('2018-07-18 22:44:00'), 'sold_price': 131.0, 'memory_size': '4GB', 'memory_type': 'GD

redirected
46
{'sold_date_time': Timestamp('2018-07-18 20:38:00'), 'sold_price': 214.49, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
47
{'sold_date_time': Timestamp('2018-07-18 20:35:00'), 'sold_price': 182.5, 'memory_size': '8GB', 'memory_type': 'GDDR5', 'manufacturer': 'AMD', 'condition': 'Used'}
48
{'sold_date_time': Timestamp('2018-07-18 20:20:00'), 'sold_price': 368.94, 'memory_size': None, 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'New'}
redirected
49
{'sold_date_time': Timestamp('2018-07-18 20:13:00'), 'sold_price': 22.0, 'memory_size': '128MB', 'memory_type': None, 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
50
{'sold_date_time': Timestamp('2018-07-18 20:11:00'), 'sold_price': 74.0, 'memory_size': '1GB', 'memory_type': 'GDDR5 SDRAM', 'manufacturer': 'AMD', 'condition': 'Used'}
redirected
51
{'sold_date_time': Timestamp('2018-07-18 20:06:00'), 'sold_price': 162.5, 'memory_size': '4GB', 'me

redirected
93
{'sold_date_time': Timestamp('2018-07-18 16:34:00'), 'sold_price': 199.99, 'memory_size': '6GB', 'memory_type': None, 'manufacturer': None, 'condition': 'Used'}
redirected
94
{'sold_date_time': Timestamp('2018-07-18 16:32:00'), 'sold_price': 399.99, 'memory_size': '8GB', 'memory_type': 'GDDR5X', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
95
{'sold_date_time': None, 'sold_price': 64.99, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'Used'}
96
{'sold_date_time': Timestamp('2018-07-18 16:30:00'), 'sold_price': 205.0, 'memory_size': None, 'memory_type': 'GDDR5', 'manufacturer': 'AMD', 'condition': 'Used'}
redirected
97
{'sold_date_time': Timestamp('2018-07-18 16:26:00'), 'sold_price': 100.0, 'memory_size': '1.5GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'New'}
redirected
98
{'sold_date_time': Timestamp('2018-07-18 16:26:00'), 'sold_price': 11.99, 'memory_size': '8MB', 'memory_type': None, 'manufacturer'

140
{'sold_date_time': Timestamp('2018-07-18 11:44:00'), 'sold_price': 59.99, 'memory_size': '1GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Seller refurbished'}
redirected
141
{'sold_date_time': None, 'sold_price': 49.99, 'memory_size': '1GB', 'memory_type': 'GDDR5', 'manufacturer': 'Nvidia GeForce GTX 645', 'condition': 'Used'}
redirected
142
{'sold_date_time': Timestamp('2018-07-18 11:29:00'), 'sold_price': 59.0, 'memory_size': '1GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
143
{'sold_date_time': Timestamp('2018-07-18 09:53:00'), 'sold_price': 7.0, 'memory_size': '128MB', 'memory_type': 'DDR2', 'manufacturer': 'ATI', 'condition': 'Used'}
144
{'sold_date_time': None, 'sold_price': 57.99, 'memory_size': '4GB', 'memory_type': 'DDR5', 'manufacturer': None, 'condition': 'New'}
145
{'sold_date_time': Timestamp('2018-07-18 09:51:00'), 'sold_price': 260.0, 'memory_size': '3 GB', 'memory_type': 'DDR5', 'manufacturer': 'AMD', 'c

191
{'sold_date_time': Timestamp('2018-07-18 03:44:00'), 'sold_price': 49.99, 'memory_size': '4GB', 'memory_type': 'DDR5', 'manufacturer': None, 'condition': 'New other (see details)'}
192
{'sold_date_time': None, 'sold_price': 43.59, 'memory_size': '1GB', 'memory_type': None, 'manufacturer': None, 'condition': 'New other (see details)'}
redirected
193
{'sold_date_time': Timestamp('2018-07-18 03:13:00'), 'sold_price': 95.0, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
194
{'sold_date_time': Timestamp('2018-07-17 02:54:00'), 'sold_price': 30.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'For parts or not working'}
redirected
195
{'sold_date_time': Timestamp('2018-07-17 02:13:00'), 'sold_price': 214.95, 'memory_size': '8GB', 'memory_type': 'GDDR5 SDRAM', 'manufacturer': 'AMD', 'condition': 'New other (see details)'}
196
{'sold_date_time': None, 'sold_price': 195.75, 'memory_size': '4GB', 

...

redirected
9772
{'sold_date_time': Timestamp('2018-06-06 08:13:00'), 'sold_price': 24.99, 'memory_size': '1GB', 'memory_type': 'DDR3', 'manufacturer': 'AMD', 'condition': 'Used'}
redirected
9773
{'sold_date_time': Timestamp('2018-06-06 07:51:00'), 'sold_price': 125.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'Used'}
redirected
9774
{'sold_date_time': None, 'sold_price': 34.0, 'memory_size': '512MB', 'memory_type': 'GDDR3', 'manufacturer': None, 'condition': 'Used'}
redirected
9775
{'sold_date_time': None, 'sold_price': 531.98, 'memory_size': '8GB', 'memory_type': None, 'manufacturer': 'NVIDIA', 'condition': 'New'}
redirected
9776
{'sold_date_time': None, 'sold_price': 159.99, 'memory_size': '1GB', 'memory_type': 'DDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9777
{'sold_date_time': Timestamp('2018-06-06 06:18:00'), 'sold_price': 224.99, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': 'AMD', 'condition': 'Used'}
re

9819
{'sold_date_time': Timestamp('2018-06-05 22:59:00'), 'sold_price': 69.99, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
9820
{'sold_date_time': Timestamp('2018-06-05 22:57:00'), 'sold_price': 500.0, 'memory_size': '8GB', 'memory_type': 'HBM2', 'manufacturer': 'AMD', 'condition': 'Used'}
redirected
9821
{'sold_date_time': Timestamp('2018-06-05 22:51:00'), 'sold_price': 11.01, 'memory_size': None, 'memory_type': 'DDR3', 'manufacturer': None, 'condition': 'Used'}
9822
{'sold_date_time': None, 'sold_price': 49.95, 'memory_size': '2GB', 'memory_type': 'DDR3', 'manufacturer': None, 'condition': 'Used'}
redirected
9823
{'sold_date_time': Timestamp('2018-06-05 22:39:00'), 'sold_price': 275.0, 'memory_size': None, 'memory_type': None, 'manufacturer': 'NVIDIA', 'condition': 'New'}
9824
{'sold_date_time': None, 'sold_price': 96.95, 'memory_size': '2GB', 'memory_type': 'DDR3', 'manufacturer': None, 'condition': 'New'}
redirected
9825
{'sold_date_

redirected
9866
{'sold_date_time': None, 'sold_price': 203.48, 'memory_size': '1.5GB', 'memory_type': 'GDDR3', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9867
{'sold_date_time': Timestamp('2018-06-05 18:39:00'), 'sold_price': 243.5, 'memory_size': '6GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9868
{'sold_date_time': Timestamp('2018-06-05 18:33:00'), 'sold_price': 39.97, 'memory_size': '1GB', 'memory_type': 'GDDR3', 'manufacturer': None, 'condition': 'Used'}
redirected
9869
{'sold_date_time': None, 'sold_price': 32.0, 'memory_size': '256MB', 'memory_type': None, 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9870
{'sold_date_time': Timestamp('2018-06-05 18:19:00'), 'sold_price': 270.0, 'memory_size': '8GB', 'memory_type': 'GDDR5', 'manufacturer': None, 'condition': 'Used'}
redirected
9871
{'sold_date_time': Timestamp('2018-06-05 18:12:00'), 'sold_price': 25.0, 'memory_size': '256MB', 'memory_type': None, 'manufacturer'

redirected
9914
{'sold_date_time': Timestamp('2018-06-05 14:25:00'), 'sold_price': 70.0, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9915
{'sold_date_time': Timestamp('2018-06-05 14:22:00'), 'sold_price': 650.0, 'memory_size': '11GB', 'memory_type': 'GDDR5X', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9916
{'sold_date_time': Timestamp('2018-06-05 14:17:00'), 'sold_price': 99.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': 'AMD', 'condition': 'Used'}
9917
{'sold_date_time': Timestamp('2018-06-05 14:07:00'), 'sold_price': 500.0, 'memory_size': '8GB', 'memory_type': 'GDDR5X', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9918
{'sold_date_time': Timestamp('2018-06-05 14:01:00'), 'sold_price': 445.0, 'memory_size': '8GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'New'}
redirected
9919
{'sold_date_time': Timestamp('2018-06-05 13:50:00'), 'sold_price': 305.0, 'memory_s

redirected
9961
{'sold_date_time': None, 'sold_price': 69.95, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9962
{'sold_date_time': Timestamp('2018-06-05 10:42:00'), 'sold_price': 225.0, 'memory_size': '4GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'New other (see details)'}
9963
{'sold_date_time': Timestamp('2018-06-05 10:21:00'), 'sold_price': 85.0, 'memory_size': '2GB', 'memory_type': 'GDDR5', 'manufacturer': 'NVIDIA', 'condition': 'Used'}
redirected
9964
{'sold_date_time': Timestamp('2018-06-05 10:11:00'), 'sold_price': 69.95, 'memory_size': None, 'memory_type': None, 'manufacturer': None, 'condition': 'Used'}
redirected
9965
{'sold_date_time': Timestamp('2018-06-05 10:04:00'), 'sold_price': 277.97, 'memory_size': '8GB', 'memory_type': 'GDDR5', 'manufacturer': 'AMD', 'condition': 'Used'}
redirected
9966
{'sold_date_time': Timestamp('2018-06-05 10:03:00'), 'sold_price': 49.99, 'memory_size': '1GB', 'mem